In [2]:
import numpy as np
import pandas as pd
import requests

from concurrent.futures import ThreadPoolExecutor
from functools import partial
import io
import urllib

import PIL.Image

from tqdm.notebook import tqdm
from multiprocessing import cpu_count

from datasets import Dataset

In [3]:
path = 'Conceptual 12M russian.csv'
# conceptual = pd.read_csv(path)

In [4]:
def fetch_single_image(image_url, timeout=None, retries=0):
    for _ in range(retries + 1):
        try:
            request = urllib.request.Request(
                image_url,
                data=None,
                headers = {
                        "User-Agent": "Googlebot-Image/1.0",  # Pretend to be googlebot
                        "X-Forwarded-For": "64.18.15.200",
                    }
#                 headers={"user-agent": USER_AGENT},
            )
            with urllib.request.urlopen(request, timeout=timeout) as req:
                image = PIL.Image.open(io.BytesIO(req.read()))
#                 print('yes')
            break
        except Exception:
            image = None
#             print('no')
    return image


def fetch_images(batch, num_threads, timeout=1, retries=0):
    fetch_single_image_with_args = partial(fetch_single_image, timeout=timeout, retries=retries)
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        batch["image"] = list(tqdm(executor.map(fetch_single_image_with_args, batch["url"]), total=len(batch)))
    return batch

In [35]:
path = 'Conceptual 12M russian.csv'
conceptual_ds = Dataset.from_csv(path, split='train[:10%]')
ds_cut2 = conceptual_ds.select(range(1001, 1003))

Generating train[:10%] split: 0 examples [00:00, ? examples/s]

KeyboardInterrupt: 

In [17]:
num_threads = 4
ds_cut = ds_cut.map(fetch_images, batched=True, batch_size=100, fn_kwargs={"num_threads": num_threads})

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
ds_cut[8]

{'url': 'https://lookaside.fbsbx.com/lookaside/crawler/media/?media_id=1496486557171364',
 'text': '\'(День 10) Команда <PERSON> получает награду BAT Grad Academy "Лучшее место для работы" на бизнес-симуляции!" Вы стремитесь работать в ведущей международной компании? Не смотрите дальше. BAT хорошо известна как одна из лучших компаний в мире для работы, сертифицированная как Top Employer во всем мире. BAT Malaysia также получила несколько наград за выдающиеся достижения в области управления персоналом, лидируя в нескольких категориях, включая «Вовлеченность сотрудников» и «Лучшие компании для работы в Азии». Это свидетельствует об инициативах и усилиях, вложенных в их повестку дня в отношении людей».',
 'image': None}

In [6]:
n_cores = cpu_count()
num_threads = 4
conceptual_ds = conceptual_ds.map(fetch_images, batched=True, batch_size=100, fn_kwargs={"num_threads": num_threads})

Map:   0%|          | 0/12423374 [00:00<?, ? examples/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [18]:
ds_cut.save_to_disk("my_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

# Download Conceptual 12M Russian

In [2]:
%pip install python-magic-bin

   ---------------------------------------- 0.0/409.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/409.3 kB ? eta -:--:--
   -- ------------------------------------ 30.7/409.3 kB 435.7 kB/s eta 0:00:01
   ----- --------------------------------- 61.4/409.3 kB 656.4 kB/s eta 0:00:01
   ---------------------- ----------------- 225.3/409.3 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------  399.4/409.3 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 409.3/409.3 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import requests
import zlib
import os
import shelve
import magic #pip install python-magic
from multiprocessing import Pool
from tqdm import tqdm


L_BOUND = 0 # TODO: попробуем частями
R_BOUND = int(1e6)

headers = {
    #'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
    'User-Agent':'Googlebot-Image/1.0', # Pretend to be googlebot
    'X-Forwarded-For': '64.18.15.200'
}

def _df_split_apply(tup_arg):
    split_ind, subset, func = tup_arg
    r = subset.apply(func, axis=1)
    return (split_ind, r)

def df_multiprocess(df, processes, chunk_size, func, dataset_name):
    print("Generating parts...")
    with shelve.open('%s_%s_%s_results.tmp' % (dataset_name, func.__name__, chunk_size)) as results:
 
        pbar = tqdm(total=len(df), position=0)
        # Resume:
        finished_chunks = set([int(k) for k in results.keys()])
        pbar.desc = "Resuming"
        for k in results.keys():
            pbar.update(len(results[str(k)][1]))

        pool_data = ((index, df[i:i + chunk_size], func) for index, i in enumerate(range(0, len(df), chunk_size)) if index not in finished_chunks)
        print(int(len(df) / chunk_size), "parts.", chunk_size, "per part.", "Using", processes, "processes")
 
        pbar.desc = "Downloading"
        with Pool(processes) as pool:
            for i, result in enumerate(pool.imap_unordered(_df_split_apply, pool_data, 2)):
                results[str(result[0])] = result
                pbar.update(len(result[1]))
        pbar.close()

    print("Finished Downloading.")
    return

# Unique name based on url
def _file_name(row):
    return "%s/%s_%s" % (row['folder'], row.name, (zlib.crc32(row['url'].encode('utf-8')) & 0xffffffff))

# For checking mimetypes separately without download
def check_mimetype(row):
    if os.path.isfile(str(row['file'])):
        row['mimetype'] = magic.from_file(row['file'], mime=True)
        row['size'] = os.stat(row['file']).st_size
    return row

# Don't download image, just check with a HEAD request, can't resume.
# Can use this instead of download_image to get HTTP status codes.
def check_download(row):
    fname = _file_name(row)
    try:
        # not all sites will support HEAD
        response = requests.head(row['url'], stream=False, timeout=5, allow_redirects=True, headers=headers)
        row['status'] = response.status_code
        row['headers'] = dict(response.headers)
    except:
        # log errors later, set error as 408 timeout
        row['status'] = 408
        return row
    if response.ok:
        row['file'] = fname
    return row

def download_image(row):
    fname = _file_name(row)
    # Skip Already downloaded, retry others later
    if os.path.isfile(fname):
        row['status'] = 200
        row['file'] = fname
        row['mimetype'] = magic.from_file(row['file'], mime=True)
        row['size'] = os.stat(row['file']).st_size
        return row

    try:
        # use smaller timeout to skip errors, but can result in failed downloads
        response = requests.get(row['url'], stream=False, timeout=5, allow_redirects=True, headers=headers)
        row['status'] = response.status_code
        #row['headers'] = dict(response.headers)
    except Exception as e:
        # log errors later, set error as 408 timeout
        row['status'] = 408
        return row
   
    if response.ok:
        try:
            with open(fname, 'wb') as out_file:
                # some sites respond with gzip transport encoding
                response.raw.decode_content = True
                print('f')
                out_file.write(response.content)
            row['mimetype'] = magic.from_file(fname, mime=True)
            row['size'] = os.stat(fname).st_size
        except:
            # This is if it times out during a download or decode
            row['status'] = 408
            return row
        row['file'] = fname
    return row

def open_tsv(fname, folder):
    print("Opening %s Data File..." % fname)
    df = pd.read_csv(fname, sep='\t', names=["url","text"])
    df = df.iloc[L_BOUND:R_BOUND]
    df['folder'] = folder
    print("Processing", len(df), " Images:")
    return df

def df_from_shelve(chunk_size, func, dataset_name):
    print("Generating Dataframe from results...")
    with shelve.open('%s_%s_%s_results.tmp' % (dataset_name, func.__name__, chunk_size)) as results:
        keylist = sorted([int(k) for k in results.keys()])
        df = pd.concat([results[str(k)][1] for k in keylist], sort=True)
    return df

In [2]:
# number of processes in the pool can be larger than cores
num_processes = 4
# chunk_size is how many images per chunk per process - changing this resets progress when restarting.
images_per_part = 100

data_name = "training"
df = open_tsv("Conceptual 12M russian.csv", data_name)

Opening Conceptual 12M russian.csv Data File...
Processing 1000000  Images:


In [3]:
# data_name = "validation"
# df = open_tsv("Validation_GCC-1.1.0-Validation.tsv", data_name)
# df_multiprocess(df=df, processes=num_processes, chunk_size=images_per_part, func=download_image, dataset_name=data_name)
# df = df_from_shelve(chunk_size=images_per_part, func=download_image, dataset_name=data_name)
# df.to_csv("downloaded_%s_report.tsv.gz" % data_name, compression='gzip', sep='\t', header=False, index=False)
# print("Saved.")

df_multiprocess(df=df, processes=num_processes, chunk_size=images_per_part, func=download_image, dataset_name=data_name)
df = df_from_shelve(chunk_size=images_per_part, func=download_image, dataset_name=data_name)
df.to_csv("downloaded_%s_report.tsv.gz" % data_name, compression='gzip', sep='\t', header=False, index=False)
print("Saved.")

Generating parts...


  0%|          | 0/1000000 [00:00<?, ?it/s]

10000 parts. 100 per part. Using 4 processes
